# Pattern Recognition & Machine Learning Project

*Authors*: Aristeidis Daskalopoulos (AEM: 10640), Georgios Rousomanis (AEM: 10703)

----

<center>
This notebook contains the solutions for Parts A, B, and C.
</center>


## Part A
### Theoretical Analysis

In this part, we address a **binary classification problem** where the goal is to classify samples into one of two classes: $\omega_1$ or $\omega_2$, based on *a single feature* $x$ (a feature vector of dimensionality one).

To achieve this, we use the probability density function (PDF) of the feature $x$, which follows - for both classes - the distribution described below:

$$ p(x|\theta) = \frac{1}{\pi}\frac{1}{1 + (x - \theta)^2} \quad, $$

where $\theta$ is an unknown parameter which has to be defined for each one of the classes separately. This PDF is the probability distribution of the [Cauchy distribution](https://en.wikipedia.org/wiki/Cauchy_distribution) for $\gamma = 1$. 

To solve this decision problem, we will implement a Generative Probabilistic Model, following the steps described in the subsequent cells.


#### 1. Maximum Likelihood Estimation (MLE)

Our first goal is to estimate the parameters $\hat{\theta}_1$ and $\hat{\theta}_2$ using the Maximum Likelihood (ML) method. To do this, we aim to maximize the log-likelihood function with respect to $\theta_j$, for $j = 1, 2$; $\theta_1$ is the parameter of the first class, and $\theta_2$ refers to class $\omega_2$.
 

Assuming that the samples $D_j$ of the class $\omega_j$, for $j = 1, 2$, are ***independent and identically distributed (i.i.d.)***, meaning they have been drawn independently from the same distribution $p(x | \theta_j, \omega_j)$, the PDF for the samples can be expressed as:

$$ p(D_j | \theta_j) = \prod_{i=1}^{N_j} p(x_i | \theta_j)\quad.$$

We prefer to work with the log-likelihood function, because it simplifies the process - as it converts multiplication into addition, which is *less* error-sensitive in terms of computational arithmetic errors (and in terms of calculating derivates). The log-likelihood of our problem is:

$$ l(\theta_j) = \log p(D_j | \theta_j) = \sum_{i=1}^{N_j} \log p(x_i | \theta_j), \quad j = 1, 2 \quad \Rightarrow$$


$$ l(\theta_j) = \sum_{i=1}^{N_j} \log \left(\frac{1}{\pi}\frac{1}{1 + (x_i - \theta_j)^2}\right), \quad j = 1, 2 \quad \Rightarrow$$

$$ l(\theta_j) = - N_j \cdot \log \pi - \sum_{i=1}^{N_j} \log (1 + (x_i - \theta_j)^2), \quad j = 1, 2 \quad,$$

with which we can estimate the $\hat{\theta}_j$ for each class. This estimate, $\hat{\theta}_j$, is by definition the value of $\theta_j$ that maximizes the likelihood/log-likelihood. In our case the term $- N_j \cdot \log \pi$ is constant, so we practically just need to *minimize* the positive term $\sum_{i=1}^{N_j} \log (1 + (x_i - \theta_j)^2)$. *It is important to keep in mind that the specific $l(\theta)$ is a negative function.*

One approach to solving this problem is to calculate the derivatives and solve the following equations, where the solution gives the estimate $\hat{\theta}_j$ for each class:

$$ \frac{d}{d\theta_j} l(\theta_j) = 0 \quad \Rightarrow \quad \frac{d}{d\theta_j} \left(- N_j \cdot \log \pi - \sum_{i=1}^{N_j} \log (1 + (x_i - \theta_j)^2)\right) = 0 \quad \Rightarrow$$

$$ \sum_{i=1}^{N_j} \frac{d}{d\theta_j}\log (1 + (x_i - \theta_j)^2) = 0 \quad \Rightarrow$$

$$ \sum_{i=1}^{N_j} \frac{-2 \cdot (x_i - \theta_j)}{1 + (x_i - \theta_j)^2} = 0 \quad \Rightarrow \quad \sum_{i=1}^{N_j} \frac{x_i - \theta_j}{1 + (x_i - \theta_j)^2} = 0 \quad.$$

*For all the $\hat{\theta}_j$ that solve the above equation we should choose the one that gives the largest (max) value to the $l(\theta_j)$.*

Given $l(\theta_j)$, its derivative can be computed efficiently (e.g., using a library like SymPy) to solve the equation and obtain the estimate $\hat{\theta}_j$. However, by plotting $l(\theta_j)$ as requested, we inherently *calculate the values of the log-likelihood function across multiple points*. Consequently, selecting the value of $\theta$ that maximizes $l(\theta)$ provides the same solution, thereby **avoiding** the need for the derivative-based approach.



#### 2. Bayes Decision Rule

Using the Bayes Decision Rule, we classify to $\omega_1$ based on the following condition:

$$ P(\omega_1 | x) > P(\omega_2 | x) \quad, $$

which can be rewritten using the *Bayes formula* as:

$$ \frac{p(x|\omega_1) P(\omega_1)}{p(x)} > \frac{p(x|\omega_2) P(\omega_2)}{p(x)} \quad, $$

or equivalently:

$$ \log p(x|\omega_1) + \log P(\omega_1) > \log p(x|\omega_2) + \log P(\omega_2) \quad. $$

*Here, the class-conditional densities $p(x|\omega_1, \theta_1)$ and $p(x|\omega_2, \theta_2)$ have been fully defined using Maximum Likelihood (ML) estimation, for the parameters $\theta_1 = \hat{\theta}_1$ and $\theta_2 = \hat{\theta}_2$ respectively.* So, for each class we have that:

$$ p(x|\omega_j) = \frac{1}{\pi}\frac{1}{1 + (x - \hat{\theta}_j)^2}, \quad P(\omega_j) = \frac{|D_j|}{|D_1| + |D_2|}, $$

where $|D_j|$ is the total number of elements, $N_j$, that this dataset has.

We define the following discriminant function:

$$ g(x) = \log p(x|\omega_1) - \log p(x|\omega_2) + \log P(\omega_1) - \log P(\omega_2) \quad, $$

and based on the previous inequity we infer that using this discriminant function:
- If $g(x) > 0$, the sample with feature $x$ is classified into class $\omega_1$.
- Otherwise, it is classified into class $\omega_2$.

The above **rule** implies that we theoretically expect the discriminant function $g(x)$ to be greater than zero when a sample from the $D_1$ set (class $\omega_1$) is provided. Based on this rule, the feature space - represented by the real number line $\mathbb{R}$ - is divided into two distinct regions: $\mathbb{R}_1$ and $\mathbb{R}_2$. To complete the theoretical analysis of this section, these regions must be defined by determining their boundaries, which can be found by solving $g(x) = 0$:


$$ \log(\frac{1}{\pi}\frac{1}{1 + (x - \hat{\theta}_1)^2}) - \log(\frac{1}{\pi}\frac{1}{1 + (x - \hat{\theta}_2)^2}) + \log(\frac{|D_1|}{|D_1| + |D_2|}) - \log(\frac{|D_2|}{|D_1| + |D_2|}) = 0 \quad \Rightarrow$$

$$ -\log(1 + (x - \hat{\theta}_1)^2) + \log(1 + (x - \hat{\theta}_2)^2) + \log(\frac{|D_1|}{|D_2|}) = 0, \quad Let\ r = \frac{|D_1|}{|D_2|} \quad \Rightarrow$$

$$ \log(\frac{1 + (x - \hat{\theta}_2)^2}{1 + (x - \hat{\theta}_1)^2}) = -\log(r) \quad \Rightarrow \quad \frac{1 + (x - \hat{\theta}_2)^2}{1 + (x - \hat{\theta}_1)^2} = \frac{1}{r} \quad \Rightarrow$$

$$ r(1 + (x - \hat{\theta}_2)^2) = 1 + (x - \hat{\theta}_1)^2 \quad \Rightarrow \quad (r-1)x^2 - 2(r\hat{\theta}_2 - \hat{\theta}_1)x + (r\hat{\theta}_2^2 + r - \hat{\theta}_1^2 - 1) = 0 $$

The solutions to this quadratic equation define the decision boundary points. These points separate regions $\mathbb{R}_1$ and $\mathbb{R}_2$ in the feature space. If the equation above has two real solutions, $x_a$ and $x_b$, then one of the regions, $\mathbb{R}_j$, will be an interval spanning $(-\infty, x_a) \cup (x_b, +\infty)$, while the other will correspond to the interval $[x_a, x_b]$. We will specify these intervals after estimating the values of $\hat{\theta}_j$.



### Algorithm Implementation

Having outlined the theoretical approach to solving this problem, we now proceed with its implementation.


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

$D_1$ and $D_2$ are the two example datasets provided. $D_1$ contains the values of the index $x$ for class $\omega_1$, while $D_2$ contains the values for class $\omega_2$. 

In the upcoming analysis, we assume that class $\omega_1$ (the positive class, where $g(x) > 0$) corresponds to the "no stress"/"relaxed" label, while class $\omega_2$ (the negative class) corresponds to the "intense stress" label. If we want to reverse this mapping, we can simply swap the $D_1$ and $D_2$ datasets' values.


In [ ]:
D1 = np.array([2.8, -0.4, -0.8, 2.3, -0.3, 3.6, 4.1])  # No stress data (class omega_1)
D2 = np.array([-4.5, -3.4, -3.1, -3.0, -2.3])     # Intense stress data (class omega_2)

Now we are ready to implement our classifier. The methods that are implemented are the following:

1. `compute_pdf`: Computes the probability density function evaluation, for given $\theta$ and $x$ values, $p(x|\theta)$.
2. `loglkhood`: Calculates the log-likelihood of a dataset $D$ given parameter $\theta$, $l(\theta) = \log p(D|\theta)$.
3. `fit`: Finds the optimal $\hat{\theta}$ parameter that maximizes the log-likelihood for the given dataset.
4. `predict`: Evaluates the discriminant function values $g(x)$ *(with which we predict the class $\omega$)* using the fitted parameters and prior probabilities. We have already explained the decision rule, which classifies samples based on the sign of the discriminant function.

More detailed information about these methods can be found in the functions' definitions.

In [ ]:
class ClassifierA:

    @staticmethod
    def compute_pdf(theta, x) -> np.ndarray:
        """
        Compute the probability density function (PDF) evaluation for given theta and x values.
        Uses the distribution formula: p(x|θ) = 1/(π(1 + (x-θ)²))
        
        For theta (M elements) and x (N elements), returns an M x N matrix where element (i,j)
        represents the PDF evaluation at x[j] for theta[i]. 
        
        Args:
            theta: Location parameter(s) of the distribution. Can be scalar or array.
            x: Data point(s) to evaluate. Can be scalar or array.
        
        Returns:
            numpy.ndarray: Matrix of PDF evaluations with shape (M, N) where M is the number
                        of theta values and N is the number of x values.
        """
        # Compute differences using broadcasting
        x, theta = map(np.atleast_1d, (x, theta))
        diff = x[None, :] - theta[:, None]

        # By simply modifying the returned PDF here, the same classifier
        # can be utilized without requiring any additional changes (for the same requirements).
        return 1.0 / (np.pi * (1.0 + diff * diff))

    @staticmethod
    def loglkhood(theta, D) -> np.ndarray:
        """
        Compute the log-likelihood of dataset D given parameter theta: l(θ) = log(p(D|θ)).
        
        For M different theta values, returns an array of M log-likelihood values.
        
        Args:
            theta: Location parameter(s) of the distribution. Can be scalar or array.
            D: Dataset for which the log-likelihood is computed. Can be scalar or array.
        
        Returns:
            numpy.ndarray: Array of log-likelihood values for each theta. 
                        Shape: (M, 1) where M is the number of theta values.
        """
        D, theta = map(np.atleast_1d, (D, theta))
        
        # Row-wise sum of the log of probabilities computed for each data point in D
        # each row corresponds to a different theta value
        return np.sum(np.log(ClassifierA.compute_pdf(theta, D)), axis=1)

    @staticmethod
    def fit(D, theta_min, theta_max, npoints=10000, plot=False, labels=0) -> float:
        """
        Find the optimal theta parameter that maximizes the log-likelihood for the given dataset.

        This function evaluates the log-likelihood for a range of theta values between
        theta_min and theta_max, using npoints for precision. The theta value corresponding to
        the maximum log-likelihood is selected as the optimal theta.

        Optionally, a plot of the log-likelihood curve can be generated. The plot includes a marker
        for the optimal theta value, with customized labels for different datasets (if specified).

        Args:
            D: Dataset for fitting the model. Can be scalar or array.
            theta_min (float): Minimum value of theta for the search range.
            theta_max (float): Maximum value of theta for the search range.
            npoints (int, optional): Number of points to sample between theta_min and theta_max. 
                                    Default is 10000.
            plot (bool, optional): If True, generates a plot of the log-likelihood curve, l(θ)=log(p(D|θ)). 
                                    Default is False.
            labels (int, optional): Dataset label for customizing plot annotations (1, 2, or others if needed). 
                                    Default is 0.

        Returns:
            float: The optimal theta value that maximizes the log-likelihood for the dataset.
        """
        theta_candidates  = np.linspace(theta_min, theta_max, npoints)
        lkhood_values     = ClassifierA.loglkhood(theta_candidates, D)
        opt_theta         = theta_candidates[np.argmax(lkhood_values)]
        
        if plot:
            # Determine appropriate label
            labels = labels if labels else ""
            
            # Plot the log-likelihood curve:
            plt.plot(theta_candidates, lkhood_values, label=rf'$\log P(D{labels}|\theta)$', 
                     color = 'blue' if labels == 1 else 'green')
            # Mark the optimal theta value on the plot:
            plt.scatter(opt_theta, ClassifierA.loglkhood(opt_theta, D), color='red', marker='x')
            
            plt.xlabel(rf'$\theta{labels}$')
            plt.ylabel('Log-likelihood')
            plt.title(f'Log-likelihood plot for Part A, $\omega{labels}$')
            plt.legend()
            plt.show()
        
        return opt_theta
    
    @staticmethod
    def predict(D, p1, p2, theta1, theta2) -> np.ndarray:
        """
        Predict class membership using the log-ratio discriminant function, g(x). 
        If theta1 is 1D array of M elements, theta2 is 1D array of K elements and 
        D is 1D array of N elements it returns a 2D array of max(M, K) x N elements 
        where the element at row i and column j is the prediction of feature D[j] corresponding 
        to theta1[i] and theta2[i] parameters.
        
        Args:
            D: Dataset points for prediction. Can be scalar or array.
            p1: Prior probability of class 1 (no stress). Must be in range (0, 1).
            p2: Prior probability of class 2 (intense stress). Must be in range (0, 1).
            theta1: Location parameter for class 1 distribution.
                Can be scalar or array of M elements.
            theta2: Location parameter for class 2 distribution.
                Can be scalar or array of K elements.
        
        Returns:
            numpy.ndarray: Discriminant function values. Shape is (max(M,K), N) where:
                        - M is the number of theta1 values
                        - K is the number of theta2 values
                        - N is the number of points in D
                        *Positive values indicate class 1, negative values indicate class 2.*
        """
        # Input validation check:
        if not 0 < p1 < 1 or not 0 < p2 < 1:
            raise ValueError("Prior probabilities must be between 0 and 1")
        if abs(p1 + p2 - 1) > 1e-5:
            raise ValueError("Prior probabilities must sum to 1")
            
        # g(x) = log(p(x|θ₁)) - log(p(x|θ₂)) + log(p₁) - log(p₂)
        return (np.log(ClassifierA.compute_pdf(theta1, D)) - 
                np.log(ClassifierA.compute_pdf(theta2, D)) + 
                np.log(p1) - np.log(p2))


#### 1. Maximum Likelihood Estimation (Results)

We execute the `fit` method for each dataset, $D_j$, to determine the optimal Maximum Likelihood estimations, $\hat{\theta}_j$. Additionally, we plot the log-likelihood function:

$$ l(\theta) = \log P(D_j | \theta), \quad \text{for} \; j = 1, 2, $$

and highlight the point where the likelihood reaches its maximum.

In [ ]:
clfA = ClassifierA()  # Create an instance of the Classifier
rng = 20
theta1 = clfA.fit(D1, -rng, rng, npoints=10000, plot=True, labels=1)
theta2 = clfA.fit(D2, -rng, rng, npoints=10000, plot=True, labels=2)
print(f'theta1 ML estimation (no stress):      {theta1}')
print(f'theta2 ML estimation (intense stress): {theta2}')

#### 2. Bayes Decision Rule (Results)

Having determined the values of $\hat{\theta}_1$ and $\hat{\theta}_2$ based on the datasets $D_1\ \text{and}\ D_2$, the next step is to evaluate whether we can correctly classify the training dataset/examples *(the classification rule we will use was already explained)*. This serves as a preliminary test to assess the model's ability to separate the classes using the learned parameters. Additionally, this step may reveal potential defects or limitations in the model.


In [ ]:
# Calculate apriori probabilities for each class:
N1 = len(D1)
N2 = len(D2)
p1 = N1 / (N1 + N2)
p2 = N2 / (N1 + N2)

# Get the discriminant values for the two classes:
predictions1 = clfA.predict(D1, p1, p2, theta1, theta2)
predictions2 = clfA.predict(D2, p1, p2, theta1, theta2)

# Scatter plot of the data points and the discriminant function values:
plt.scatter(D1, predictions1, label='no stress', color='blue', marker='o')
plt.scatter(D2, predictions2, label='intense stress', color='green', marker='x')
# Add a horizontal dashed line (threshold for classification):
plt.axhline(y=0.0, color='red', linestyle='--', label="threshold")

plt.xlabel('x')
plt.ylabel('g(x)')
plt.title('Discriminant function values for D1, D2 datasets')
plt.legend()
plt.show()

From the above plot, we observe only one misclassification, where a sample that should have been classified as "no stress" was incorrectly predicted as "intense stress." Additionally, it is evident that some "no stress" values are near the threshold.  

We should now implement a validation method to ensure that the classifier can correctly classify samples it has ***never seen before***. It is important to note that, due to the very limited number of training samples, splitting $D_1 \cup D_2$ into training and validation sets may not yield reliable results. To address this, we will use *Leave-One-Out Cross-Validation* (LOOCV) to evaluate the accuracy, precision, and recall of our model. This approach ensures that every sample is used both for training and as a validation point at least once. 

By employing LOOCV, we can verify that our approach is truly learning the underlying patterns (and not merely memorizing the training set). Additionally, this provides a consistent metric for *comparison with the results obtained in Part B.*


In [ ]:
# Prepare labeled dataset:
D1_labeled = np.column_stack((D1, np.ones(len(D1))))      # Class 1 (positive g(x) - relaxed)
D2_labeled = np.column_stack((D2, 2 * np.ones(len(D2))))  # Class 2 (negative g(x) - stressed)
loocv_set = np.concatenate([D1_labeled, D2_labeled])

clfML = ClassifierA()
true_positives  = 0  # Correctly predicted class 1
false_positives = 0  # Predicted class 1 when actually class 2
false_negatives = 0  # Predicted class 2 when actually class 1
true_negatives  = 0  # Correctly predicted class 2

for i in range(len(loocv_set)):
    # Create training set excluding current test instance:
    train_set = np.concatenate([loocv_set[:i], loocv_set[i+1:]])
    
    D1_train_values = train_set[train_set[:, 1] == 1][:, 0]
    D2_train_values = train_set[train_set[:, 1] == 2][:, 0]
    
    theta_1 = clfML.fit(D1_train_values, -10, +10)
    theta_2 = clfML.fit(D2_train_values, -10, +10)
    
    N1 = len(D1_train_values)
    N2 = len(D2_train_values)
    p_1 = N1 / (N1 + N2)
    p_2 = N2 / (N1 + N2)
    
    test_instance =  loocv_set[i][0]
    true_label    =  loocv_set[i][1]
    prediction = clfML.predict(test_instance, p_1, p_2, theta_1, theta_2)
    
    if prediction > 0:  # Predicted class 1
        if true_label == 1:
            true_positives += 1
        else:
            false_positives += 1
    else:               # Predicted class 2
        if true_label == 1:
            false_negatives += 1
        else:
            true_negatives += 1

# Calculate metrics
accuracy  =  (true_positives + true_negatives) / len(loocv_set)
precision =  true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
recall    =  true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
f1_score  =  2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"Accuracy:  {accuracy}")
print(f"Precision: {precision}")
print(f"Recall:    {recall}")
print(f"F1 Score:  {f1_score}")

The above metrics reflect the performance of the classifier, where:

- $\omega_1$: "relaxed" label (positive)
- $\omega_2$: "stressed" label (negative)

1. **Accuracy**: $0.833$  
   Accuracy measures the overall correctness of the classifier by calculating the proportion of correctly classified samples out of all samples.

2. **Precision**: $1.0$  
   Precision focuses on the classifier's ability to correctly identify positive samples (labeled as $\omega_1$, "relaxed"). A precision of $1.0$ means that all samples predicted as "relaxed" were indeed "relaxed," with no false positives.

3. **Recall**: $0.714$  
   Recall, also known as *sensitivity*, measures the classifier's ability to correctly detect all actual "relaxed" samples. A recall of $0.714$ indicates that 71.43% of the true "relaxed" samples were identified, meaning some were missed (false negatives).

4. **F1 Score**: $0.833$  
   The F1 Score is the harmonic mean of precision and recall, providing a balanced measure of the classifier's performance. An F1 Score of $0.833$ reflects strong performance, though it highlights the trade-off between perfect precision and ***suboptimal recall***.

*Note*: In the previous analysis, without domain knowledge of the game, we focused on the "relaxed/no stress" label. This decision was based on our observation that the model tends to make more errors when predicting this label. Despite these challenges, the overall accuracy remains acceptable, considering the very limited number of examples available in our dataset.



***Lastly***, based on the previously evaluated $\hat{\theta}_j$ results, we can **solve the equation:** $g(x) = 0$ and determine the two regions, $\mathbb{R}_1$ and $\mathbb{R}_2$, in the feature space. By determining these regions, we would have *fully explained the classification rule*, whose accuracy on the dataset has *already* been measured using LOOCV.

We have found that $\hat{\theta}_1 \simeq 2.6$ and $\hat{\theta}_2 \simeq -3.16$. To solve the quadratic equation:

$$ (r-1)x^2 - 2(r\hat{\theta}_2 - \hat{\theta}_1)x + (r\hat{\theta}_2^2 + r - \hat{\theta}_1^2 - 1) = 0, $$

where $r = \frac{7}{5}$, we can apply the quadratic formula and get the roots:

$$ x_a \approx −34.57, \quad x_b \approx −0.55.$$

These values define the boundaries of the regions $\mathbb{R}_1$ and $\mathbb{R}_2$ in the feature space.

Now, we proceed to validate the intervals $\mathbb{R}_1$ and $\mathbb{R}_2$ by evaluating the $g(x)$ with specific values and checking the sign of the outcomes.This way, we will ensure that the intervals match those obtained theoretically:


In [ ]:
xValues = np.linspace(-50, 10, 100000)
predictions = clfA.predict(xValues, p1, p2, theta1, theta2)

if len(predictions.shape) > 1:
    predictions = predictions[0]  # Take first row if 2D array

plt.figure(figsize=(12, 2))
# Plot points with positive predictions in red:
plt.plot(xValues[predictions > 0], np.zeros_like(xValues[predictions > 0]), 'ro', 
         label='Class ω₁', markersize=1)
# Plot points with negative predictions in blue:
plt.plot(xValues[predictions <= 0], np.zeros_like(xValues[predictions <= 0]), 'bo', 
         label='Class ω₂', markersize=1)

plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
plt.grid(True, axis='x')
plt.title('Decision Regions on the Real Line')
plt.xlabel('x')
plt.yticks([])
plt.legend()
plt.show()

x_a = xValues[predictions <= 0][0]
x_b = xValues[predictions <= 0][len(xValues[predictions <= 0]) - 1]

print(f"R1 interval: (-inf, {x_a}) and ({x_b}, +inf) --> class ω₁")
print(f"R2 interval: [{x_a}, {x_b}]                  --> class ω₂")

## Part B
### Theoretical Analysis

In the earlier section, we worked with a limited dataset to classify samples using the *Maximum Likelihood* estimation for the parameter $\theta$ for each class. The classification was performed based on the feature "vector" (has a dimensionality of one) $x \in \mathbb{R}$. 

Now, we extend our approach by incorporating the *prior knowledge* of the probability distribution $p(\theta)$. This allows us to make a more refined estimate of the *posterior* distribution of $\theta$. We anticipate that the density of this posterior distribution will be ***sharply concentrated around the true value*** of $\theta$, thereby improving the accuracy and reliability of our model.


#### 1. Bayesian Estimation: Posterior PDF of $\theta$

Our goal now is to refine our estimation of the parameter $\theta$, for each class, by leveraging both *prior knowledge* and *observed data*. By incorporating the prior PDF of $\theta$, we compute the posterior distribution of $\theta$ given the data. This posterior distribution provides a more precise and informed estimate of $\theta$, shaped around its true value. 
 
The **prior PDF** of $\theta$ is defined as:  

$$ p(\theta_j) = \frac{1}{10\pi} \frac{1}{1 + (\theta_j / 10)^2}, \quad j = 1, 2 \quad,$$

and given the dataset $D_j$ for class $\omega_j$, the **likelihood function** is computed as:  

$$ p(D_j|\theta_j) = \prod_{n=1}^{N_j} p(x_n|\theta_j), \quad j = 1, 2 \quad, $$

where, $N_j$ denotes the number of samples in $D_j$, and $p(x_n|\theta_j)$ represents the likelihood of observing the feature $x_n$ given the parameter $\theta_j$.
 
Using Bayes' theorem, the **posterior PDF** of $\theta$ is given by:  

$$ p(\theta_j|D_j) = \frac{p(D_j|\theta_j) p(\theta_j)}{\int p(D_j|\theta_j) p(\theta_j) \, d\theta_j}, \quad j = 1, 2 \quad. $$


#### 2. Bayesian Estimation: Decision

Using the *Bayesian Estimation*, we classify to $\omega_1$ based on the following condition:

$$ p(\omega_1 | x, D_1) > p(\omega_2 | x, D_2) $$

which also can be rewritten using the *Bayes formula* as:

$$ 
\frac{p(x | D_1) P(\omega_1)}{p(x | D_1) P(\omega_1) + p(x | D_2) P(\omega_2)} > 
\frac{p(x | D_2) P(\omega_2)}{p(x | D_1) P(\omega_1) + p(x | D_2) P(\omega_2)}
$$

or

$$ \log p(x | D_1) + \log P(\omega_1) > \log p(x | D_2) + \log P(\omega_2) \quad. $$

By selecting the following discriminant function:

$$ h(x) = \log p(x | D_1) - \log p(x | D_2) + \log P(\omega_1) - \log P(\omega_2) \quad, $$

we once again classify an element/sample with feature value $x$ into class $\omega_1$ if $h(x) > 0$, and into class $\omega_2$ otherwise. *This same classification **rule** was applied in Part A*. However, this time we need to determine the **class conditional density** $p(x | D_j)$.

We have already demonstrated how to compute the posterior density $p(\theta_j | D_j)$. Using this, along with the feature PDF $p(x | \theta_j)$, we can evaluate the *marginal density* $p(x | D_j)$ by integrating the joint density $p(x, \theta | D_j)$ over $\theta$:

$$ p(x | D_j) = \int p(x, \theta | D_j) \, d\theta. $$

By applying the product rule and because the selection of
$x$ and $D_j$ are done *independently*, this equation can be rewritten as:

$$ p(x | D_j) = \int_{-\infty}^{+\infty} p(x | \theta) p(\theta | D_j) \, d\theta. $$

The above integral can be easily computed using the ***trapezoidal rule***. Instead of letting $\theta$ range from $-\infty$ to $+\infty$, we can limit the integration to two sufficiently large bounds. These bounds are chosen to *cover all the significant non-zero areas* of the distributions. For very large or very small values of $\theta$, the product of the distributions becomes nearly zero. As a result, these values have negligible effect on the overall computation.



### Algorithm Implementation

Now we are ready to implement our second classifier which uses a ***Bayesian approach***, where instead of finding a point estimate for $\theta$, we compute the *full posterior distribution* and use it for predictions through marginalization. By taking into account this information the classifier leads to better solutions (we will see this in the validation section) than the simple ML approach of part A. 

The methods that are implemented are the following:

1. `compute_feature_pdf`: Computes the probability density function $p(x|\theta) = \frac{1}{\pi(1 + (x-\theta)^2)}$ for given $\theta$ and $x$ values
2. `prior_theta_pdf`: Computes the prior probability density $p(\theta) = \frac{1}{10\pi(1 + (\theta/10)^2)}$ of the parameter $\theta$
3. `compute_lkhood`: Calculates the likelihood $L(\theta|D) = p(D|\theta) = \prod p(x|\theta)$ for $x \in D$
4. `posterior_theta_pdf`: Computes the posterior probability density $p(\theta|D) \propto p(D|\theta)p(\theta)$ using Bayes' theorem
5. `compute_marginal_feature_pdf`: Evaluates the marginal PDF $p(x|D) = \int p(x|\theta)p(\theta|D)d\theta$ by integrating over $\theta$
6. `predict`: Predicts the class $\omega$ by evaluating the discriminant function $g(x) = \log(p(x|D_1)) - \log(p(x|D_2)) + \log(p_1) - \log(p_2)$

*More detailed information about these methods can be found in the functions definitions.*


In [ ]:
class ClassifierB:

    @staticmethod
    def compute_feature_pdf(theta, x) -> np.ndarray:
        """
        Compute the probability density function (PDF) evaluation for given theta and x values.
        Uses the distribution formula: p(x|θ) = 1/(π(1 + (x-θ)²)).
        
        For theta (M elements) and x (N elements), returns an M x N matrix where element (i,j)
        represents the PDF evaluation at x[j] for theta[i].
        
        Args:
            theta: Location parameter(s) of the distribution. Can be scalar or array.
            x: Data point(s) to evaluate. Can be scalar or array.
        
        Returns:
            numpy.ndarray: Matrix of PDF evaluations with shape (M, N) where M is the number
                        of theta values and N is the number of x values.
        """
        # Ensure at least 1D:
        x, theta = map(np.atleast_1d, (x, theta))
        diff = x[None, :] - theta[:, None]
        
        # Compute PDF: p(x|θ) = 1/(π(1 + (x-θ)²))
        return 1.0 / (np.pi * (1.0 + diff * diff))

    @staticmethod
    def prior_theta_pdf(theta) -> np.ndarray:
        """
        Compute the prior PDF of theta.
        Uses the formula: p(θ) = 1/(10π(1 + (θ/10)²))
        
        For theta (M elements), returns an array of M elements containing the prior
        probability evaluations for each theta value.
        
        Args:
            theta: Parameter values to evaluate. Can be scalar or array.
        
        Returns:
            numpy.ndarray: Array of prior probability values for each theta.
        """
        theta = np.atleast_1d(theta)
        return 1.0 / (10.0 * np.pi * (1.0 + (theta / 10.0) ** 2))

    @staticmethod
    def compute_lkhood(theta, D) -> np.ndarray:
        """
        Compute the likelihood of dataset D given parameter theta.
        Uses the formula: L(θ|D) = p(D|θ) = ∏ p(x|θ), for x in D.
        
        For theta (M elements) and dataset D, returns an array of M likelihood values
        computed as the product of individual probabilities p(x|θ) for all x in D.
        
        Args:
            theta: Parameter values to evaluate. Can be scalar or array.
            D: Dataset points for likelihood computation. Can be scalar or array.
        
        Returns:
            numpy.ndarray: Array of likelihood values for each theta parameter.
        """
        # Compute product of probabilities across all data points for each theta
        return np.prod(ClassifierB.compute_feature_pdf(theta, D), axis=1)
    
    @staticmethod
    def posterior_theta_pdf(theta, D, theta_min=-1000, theta_max=1000, npoints=10000) -> np.ndarray:
        """
        Compute the posterior PDF using Bayes' theorem.
        Uses Bayes' theorem: p(θ|D) ~ p(D|θ)p(θ), normalized by integration over θ.
        
        For theta (M elements), returns an array of M elements containing the posterior
        probability evaluations for each theta value.
        
        Args:
            theta: Parameter values to evaluate. Can be scalar or array.
            D: Dataset points for posterior computation. Can be scalar or array.
        
        Returns:
            numpy.ndarray: Array of posterior probability values for each theta.
        """
        # Compute normalization integral
        theta_range = np.linspace(theta_min, theta_max, npoints)
        unnormalized_posterior = (ClassifierB.compute_lkhood(theta_range, D) * 
                                ClassifierB.prior_theta_pdf(theta_range))
        normalization = np.trapz(unnormalized_posterior, theta_range)
        
        # Compute normalized posterior
        return (ClassifierB.compute_lkhood(theta, D) * 
                ClassifierB.prior_theta_pdf(theta) / normalization)
    
    @staticmethod
    def compute_marginal_feature_pdf(x, D, theta_min=-1000, theta_max=1000, npoints=10000) -> np.ndarray:
        """
        Compute the marginal PDF p(x|D) by integrating over theta.
        
        For each x value, computes p(x|D) = ∫ p(x|θ)p(θ|D)dθ using numerical integration.
        The integration is performed over a finite range of theta values.
        
        Args:
            x: Points at which to evaluate the marginal PDF. Can be scalar or array.
            D: Dataset used for computing the posterior. Can be scalar or array.
        
        Returns:
            numpy.ndarray: Array of marginal PDF values for each x point.
        """
        theta_values = np.linspace(theta_min, theta_max, npoints)
        
        # Compute p(θ|D):
        posterior = ClassifierB.posterior_theta_pdf(theta_values, D)[:, np.newaxis]
        # Compute integrand p(x|θ)p(θ|D) for each theta and x:
        integrand = posterior * ClassifierB.compute_feature_pdf(theta_values, x)
        
        # Integrate over theta to get marginal p(x|D):
        return np.trapz(integrand, theta_values, axis=0)

    @staticmethod
    def predict(D, D1, D2, p1, p2) -> np.ndarray:
        """
        Predict class membership using the marginal likelihood ratio discriminant function.
        
        Computes g(x) = log(p(x|D₁)) - log(p(x|D₂)) + log(p₁) - log(p₂) for each x in D,
        where positive values indicate class 1 and negative values indicate class 2.
        
        Args:
            D: Data points for prediction. Can be scalar or array.
            D1: Training dataset for class 1 (no stress).
            D2: Training dataset for class 2 (intense stress).
            p1: Prior probability of class 1. Must be in range (0, 1).
            p2: Prior probability of class 2. Must be in range (0, 1).
        
        Returns:
            numpy.ndarray: Discriminant function values for each point in D.
                        Positive values indicate class 1, negative values indicate class 2.
        """
        # Input validation for prior probabilities
        if not 0 < p1 < 1 or not 0 < p2 < 1:
            raise ValueError("Prior probabilities must be between 0 and 1")
        if abs(p1 + p2 - 1) > 1e-5:
            raise ValueError("Prior probabilities must sum to 1")
        
        return (np.log(ClassifierB.compute_marginal_feature_pdf(D, D1)) - 
                np.log(ClassifierB.compute_marginal_feature_pdf(D, D2)) + 
                np.log(p1) - np.log(p2))

#### 1. Bayesian Estimation: Posterior PDF of $\theta$ (Results)

To determine the posterior PDF of $\theta$, we execute the `posterior_theta_pdf` method for each dataset $D_j$. We also evaluate the *Maximum A Posteriori* (MAP) estimates of $\theta_1$ and $\theta_2$. 

We expect these MAP estimates to closely ***match*** (or *at least be near to*) the results obtained from the Maximum Likelihood (ML) estimations of $\theta_j$. Additionally, we compute the prior PDF of $\theta$, using the `prior_theta_pdf` method, and plot all the results on the same diagram for comparison.


In [ ]:
clfB = ClassifierB()  # Create an instance of the Classifier of Part B.
theta_max = 40  # Range limit for theta
npoints = 1000  # Number of points to plot
theta_values = np.linspace(-theta_max, theta_max, npoints)  # Theta range for plotting

# Compute probability densities
prior_density = clfB.prior_theta_pdf(theta_values)  # Prior pdf values for theta
posterior_density_class1 = clfB.posterior_theta_pdf(theta_values, D1)  # Posterior pdf values for theta given dataset D1
posterior_density_class2 = clfB.posterior_theta_pdf(theta_values, D2)  # Posterior pdf values for theta given dataset D2

# Find MAP (Maximum A Posteriori) estimates
idx1 = np.argmax(posterior_density_class1)  # Index of maximum posterior pdf given D1
idx2 = np.argmax(posterior_density_class2)  # Index of maximum posterior pdf given D2

#################### Print MAP estimates #####################
print(f'MAP estimate of theta1 (no stress):      {theta_values[idx1]}')  # Print MAP estimate for class 1
print(f'MAP estimate of theta2 (intense stress): {theta_values[idx2]}')  # Print MAP estimate for class 2

################# Plot probability densities #################
plt.plot(theta_values, prior_density, label=r'$p(\theta)$', color='red')
plt.plot(theta_values, posterior_density_class1, label=r'$p(\theta|D1)$', color='blue')
plt.plot(theta_values, posterior_density_class2, label=r'$p(\theta|D2)$', color='green')

# Mark MAP points
plt.scatter(theta_values[idx1], posterior_density_class1[idx1], color='blue', marker='x')
plt.scatter(theta_values[idx2], posterior_density_class2[idx2], color='green', marker='x')

plt.xlabel(r'$\theta$')
plt.ylabel('Probability Density')
plt.title(r'Prior & posterior density functions of $\theta$')
plt.legend()
plt.show()

The posterior density functions $p(\theta | D_j)$ are sharply concentrated around their respective *MAP estimates*, which closely align with the *Maximum Likelihood (ML) estimates*. This indicates that the posterior distributions provide confident estimates of $\theta$, *as informed by the datasets $D_j$*. 

In contrast, the prior $p(\theta)$ is a much wider distribution, offering less confidence in its estimates when considered alone. As a result, ***the posterior is primarily informed*** by the data in the datasets $D_j$, with respect to the prior $p(\theta)$ which also influences the final results.

To assess whether the knowledge of the prior $p(\theta)$ has a significant impact, we will once again perform LOOCV for the new classifier and evaluate the resulting performance metrics.


#### 2. Bayesian Estimation: Decision (Results)

Before performing LOOCV, we will first test the model's ability to correctly classify the training datasets $D_j$. This serves as a preliminary evaluation to verify whether the model can effectively distinguish between the classes, using the learned parameters. It may also reveal potential flaws in our model.


In [ ]:
# Get the discriminant values for the two classes
predictions1 = clfB.predict(D1, D1, D2, p1, p2)
predictions2 = clfB.predict(D2, D1, D2, p1, p2)

# Scatter plot of the data points and the discriminant function values
plt.scatter(D1, predictions1, label='no stress', color='blue', marker='o')
plt.scatter(D2, predictions2, label='intense stress', color='green', marker='x')

# Add a horizontal dashed line (threshold for classification)
plt.axhline(y=0.0, color='red', linestyle='--', label="threshold")

# Labeling the plot
plt.xlabel('x')
plt.ylabel('h(x)')
plt.title('Discriminant function values for D1, D2 datasets')
plt.legend()
plt.show()


It is evident that Bayesian Estimation gives us better results than ML estimation, because this time $h(x) > 0$ for all the $D_1$ dataset, with no values of $h(x)$ "too close" to the threshold (they are further than the previous classifier). This is beacause BE takes into account the prior distribution of the $\theta$ parameter, leading to better solutions.

To confirm the validity of this finding, we will validate our model using the same datasets $D_j$, applying the LOOCV method *as done in Part A*. This will allow us to assess the model's ability to predict unseen samples. 

By comparing the resulting performance metrics, we can gain a clearer understanding of the superiority of the current classifier (B) over the previous one.


In [ ]:
# Prepare labeled dataset:
D1_labeled = np.column_stack((D1, np.ones(len(D1))))      # Class 1 (positive g(x) - relaxed)
D2_labeled = np.column_stack((D2, 2 * np.ones(len(D2))))  # Class 2 (negative g(x) - stressed)
loocv_set = np.concatenate([D1_labeled, D2_labeled])

clfBE = ClassifierB()
true_positives  = 0  # Correctly predicted class 1
false_positives = 0  # Predicted class 1 when actually class 2
false_negatives = 0  # Predicted class 2 when actually class 1
true_negatives  = 0  # Correctly predicted class 2

for i in range(len(loocv_set)):
    # Create training set excluding current test instance:
    train_set = np.concatenate([loocv_set[:i], loocv_set[i+1:]])
    
    D1_train_values = train_set[train_set[:, 1] == 1][:, 0]
    D2_train_values = train_set[train_set[:, 1] == 2][:, 0]
    
    N1 = len(D1_train_values)
    N2 = len(D2_train_values)
    p_1 = N1 / (N1 + N2)
    p_2 = N2 / (N1 + N2)
    
    test_instance =  loocv_set[i][0]
    true_label    =  loocv_set[i][1]
    prediction = clfBE.predict(test_instance, D1_train_values, D2_train_values, p_1, p_2)
    
    if prediction > 0:  # Predicted class 1
        if true_label == 1:
            true_positives += 1
        else:
            false_positives += 1
    else:               # Predicted class 2
        if true_label == 1:
            false_negatives += 1
        else:
            true_negatives += 1

# Calculate metrics
accuracy  =  (true_positives + true_negatives) / len(loocv_set)
precision =  true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
recall    =  true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
f1_score  =  2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"Accuracy:  {accuracy}")
print(f"Precision: {precision}")
print(f"Recall:    {recall}")
print(f"F1 Score:  {f1_score}")

The improved results demonstrate **the advantage of incorporating the prior** $p(\theta)$ into the model. The higher accuracy (91.67%) and F1 score (92.31%) indicate that the classifier is better at distinguishing between classes, while we still have precision 1.0. The increase in recall (from 71.43% to 85.71%) suggests that the model is capturing ~20% more true positives compared to the previous approach, leading to a more reliable classification.


## Part C

### Section 1: Decision Tree Classifier

#### 1. Tuning depth hyperparameter for Decision Tree Classifier

To evaluate the performance of a Decision Tree classifier on the *Iris* dataset for varying tree depths we do the following:

1. **Loading the Dataset**: Using the first two features for simplicity and the target labels.

2. **Splitting Data**: Dividing the dataset into training and testing sets (50% each) for model evaluation.

3. **Testing Tree Depths**: Iteratively training a Decision Tree classifier with depths ranging from 1 to 100.

4. **Calculating Accuracy**: For each tree depth, we compute the accuracy of the model on the test set.

5. **Finding Optimal Depth**: Identifying the tree depth that results in the highest accuracy, which balances model 
complexity and performance.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import plot_tree  # for visualizing the decision tree structure
from matplotlib.colors import ListedColormap

In [ ]:
iris = load_iris()  # Load the Iris dataset

X = iris.data[:, :2]  # Extract the first two features of the dataset
y = iris.target  # Get the target values of the dataset

rnd_seed = 42  # Random seed for reproducibility

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=rnd_seed)

In [ ]:
max_depth = 100  # Maximum depth of the tree
depths = np.arange(1, max_depth + 1)  # Range of depths

accuracies_DT = np.zeros(max_depth)  # accuracy achieved for each depth of the decision tree

# Test the accuracy of the DT for different tree depths
for depth in depths:
    # Create an instance of the classifier
    clf = DecisionTreeClassifier(max_depth=depth, random_state=rnd_seed)
    clf.fit(X_train, y_train)  # Train the classifier with the training data
    y_pred = clf.predict(X_test)  # Find predictions of the model for the test set
    accuracies_DT[depth - 1] = accuracy_score(y_test, y_pred)  # Calculate accuracy of the model

best_depth_DT = np.argmax(accuracies_DT) + 1  # Find the depth of the tree that gives the best accuracy
best_accuracy_DT = accuracies_DT[best_depth_DT - 1]  # Find the best accuracy
print(f'Decision Tree: Best depth={best_depth_DT}, Accuracy={best_accuracy_DT}')

In [ ]:
plt.plot(depths, accuracies_DT, color='red')
plt.title('Accuracy vs Depth for Decision Tree Classifier')
plt.xlabel('Depth')
plt.ylabel('Accuracy')
plt.show()

**Key observations**

- The accuracy initially increases as the tree depth grows, reaching a peak at a certain depth (depth 3 in this case).

- Beyond that point, the accuracy declines and stabilizes. This suggests overfitting: deeper trees memorize the training data but fail to generalize to unseen data.

- The highest accuracy occurs at a relatively shallow depth. This indicates that the shallower tree offers a balance 
between bias and variance, leading to better generalization.

In the next steps we will see how we can further improve the performance using ensemble methods (e.g., Random Forest).

#### 2. Decision Regions for Decision Tree Classifier with optimal depth

To visualize the decision boundaries of the *Decision Tree Classifier with optimal depth* we do the following:

1. **Model Training:** We train the model with the best depth with the initial training data, 
that is the training dataset used in the previous step.

2. **Meshgrid Creation:** We generate a dense grid of points spanning the feature space to visualize decision boundaries. 

3. **Prediction for Grid Points:** Each point on this grid is classified by the model, assigning it a predicted class.

4. **Color Mapping:** Using a contour plot, we map each region with the same predicted class to a unique color, effectively highlighting the decision regions and illustrating how the classifier partitions the feature space.

5. **Plot Decision Tree Structure:** As an extra step, we visualize the structure of the trained decision tree i.e.
the feature selected to maximize the class seperation at each step, the corresponding gini index etc.

In [ ]:
# Create a meshgrid to plot decision boundaries
npoints = 1000
x_min, x_max = X[:, 0].min(), X[:, 0].max()  # Define x-axis range
y_min, y_max = X[:, 1].min(), X[:, 1].max()  # Define y-axis range
x_margin = 0.1 * (x_max - x_min)  # Define x-axis margin
y_margin = 0.1 * (y_max - y_min)  # Define y-axis margin
xx, yy = np.meshgrid(np.linspace(x_min - x_margin, x_max + x_margin, npoints), 
                     np.linspace(y_min - y_margin, y_max + y_margin, npoints))

# Create an instance of the classifier with the optimal tree depth
clf = DecisionTreeClassifier(max_depth=best_depth_DT, random_state=rnd_seed)
clf.fit(X_train, y_train)  # Train the classifier with the training data

# Predict the class for each point in the grid
predictions = clf.predict(np.c_[xx.ravel(), yy.ravel()])  # Use grid points as input
predictions = predictions.reshape(xx.shape)  # Reshape predictions to match the grid's shape

# Create a ListedColormap for the contour plot
custom_cmap = ListedColormap(['#FF9999', '#99FF99', '#9999FF'])

# Plot the decision boundaries
plt.contourf(xx, yy, predictions, alpha=0.8, cmap=custom_cmap)

# Plot the training and testing points
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, marker='s', edgecolor='k', cmap=custom_cmap, label='Train')
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, marker='o', edgecolor='k', cmap=custom_cmap, label='Test')

plt.title(f'Decision Boundaries for DT (depth: {best_depth_DT})')
plt.xlabel(iris.feature_names[0])
plt.ylabel(iris.feature_names[1])
plt.legend()
plt.show()

In [ ]:
# Visualize the structure of the decision tree
plt.figure(figsize=(12, 8))
plot_tree(clf, feature_names=iris.feature_names[:2], class_names=iris.target_names, filled=True)
plt.show()

**Key observations**

- The model performs well in distinguishing the classes striking a balance between complexity and generalization.

- Decision regions are not too complex memorizing training data, avoiding overfitting.

- The boundaries are axis-aligned due to the nature of Decision Trees (splits occur along a single feature at each 
decision point).

- The decision boundaries align with the splits seen in the tree visualization (the number of horizontal and vertical
boundaries are equal to the number of splits that lead to seperation into different class regions).

- The gini index tends to decrease as we go down the tree, since DT aims to split the data at each node to 
maximize class separation (reduce impurity).

### Section 2: Random Forest Classifier

In this section, we illustrate the advantages of ensemble learning and, in particular, Random Forests over single Decision 
Trees. Random Forests reduce overfitting by combining the predictions of multiple trees, ensuring better generalization to 
unseen data. Additionally, they are more robust to noise and outliers, as the ensemble approach smooths out the variability
of individual trees.

#### 1. Tuning tree depth hyperparameter for Random Forest Classifier

The procedure is the same as in the case of *Decision Tree Classifier*.

In [ ]:
n_trees = 100  # Number of trees
gamma = 0.5  # Fraction of the original training data to use for bootstrap sampling
accuracies_RF = np.zeros(max_depth)  # Accuracy achieved by the Random Forest Classifier for each tree depth

# Test the accuracy of the RF for different tree depths
for depth in depths:
    # Create an instance of the RF classifier
    clf = RandomForestClassifier(n_estimators=n_trees, max_depth=depth, random_state=rnd_seed, bootstrap=True, max_samples=gamma, n_jobs=-1)
    clf.fit(X_train, y_train)  # Train the classifier
    y_pred = clf.predict(X_test)  # Make predictions
    accuracies_RF[depth - 1] = accuracy_score(y_test, y_pred)  # Compute the accuracy

best_depth_RF = np.argmax(accuracies_RF) + 1  # Find the depth of the tree that gives the best accuracy for RF
best_accuracy_RF = accuracies_RF[best_depth_RF - 1]  # Find the best accuracy for RF
print(f'Random Forest: Best depth={best_depth_RF}, Accuracy={best_accuracy_RF}')

In [ ]:
# Plot the accuracy versus the depth of the tree for DT and RF
plt.plot(depths, accuracies_DT, label='DT', color='red')
plt.plot(depths, accuracies_RF, label='RF', color='blue')
plt.title('Accuracy vs Depth for DT & RF')
plt.xlabel('Depth')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

**Key observations**

- The Random Forest (RF) shows higher accuracy compared to the Decision Tree (DT) across all depth values.
Averaging predictions of weak learners, such as trees, trained on different datasets leads to reduction of the variance
and therefore better accuracy.

- As in the case of DT, the accuracy initially increases as the tree depths grow, reaching a peak at depth 2.

- Beyond that point the significant drop in the accuracy is an indication of overfitting.

- The Random Forest performs better and generalizes more effectively than the Decision Tree, particularly at higher depths,
due to ensemble averaging.

#### 2. Decision Regions for Random Forest Classifier with optimal depth

In [ ]:
# Create an instance of the random forest classifier with the optimal depth
clf = RandomForestClassifier(n_estimators=n_trees, max_depth=best_depth_RF, random_state=rnd_seed, bootstrap=True, max_samples=gamma, n_jobs=-1)
clf.fit(X_train, y_train)  # Train the classifier
predictions = clf.predict(np.c_[xx.ravel(), yy.ravel()])  # Make predictions, use as input the grid points
predictions = predictions.reshape(xx.shape)  # Reshape predictions to match the grid's shape

# Plot the decision boundaries
plt.contourf(xx, yy, predictions, alpha=0.8, cmap=custom_cmap)

# Plot the training and testing points
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, marker='s', edgecolor='k', label='Train', cmap=custom_cmap)
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, marker='o', edgecolor='k', label='Test', cmap=custom_cmap)

plt.title(f'Decision Boundaries for RF (depth: {best_depth_RF})')
plt.legend()
plt.show()

**Key observations**

- The decision boundaries for RF (depth = 2) appear smoother and more generalized compared to those of the individual DT 
(depth = 3). This is a result of the ensemble method (averaging decisions of multiple trees), which reduces variance and produces more balanced boundaries.

- The decision boundaries accurately reflect the overall structure of the data and remain robust to the influence of 
outliers.

#### 3. Accuracy versus $\gamma$ for Random Forest Classifier 

In [ ]:
gammas = np.array([0.1, 0.3, 0.5, 0.8, 1.0])  # gamma parameters
accuracies_RF = np.zeros((gammas.size, max_depth))

# Test the accuracy of the RF for different values of the gamma parameter
# for fixed tree depth equal to the optimal
for i in range(0, gammas.size):
        for j in range(0, max_depth):
            # Create an instance of the RF classifier
            clf = RandomForestClassifier(n_estimators=n_trees, max_depth=j+1, random_state=rnd_seed, bootstrap=True, max_samples=gammas[i], n_jobs=-1)
            clf.fit(X_train, y_train)  # Train the classifier
            y_pred = clf.predict(X_test)  # Make predictions
            accuracies_RF[i][j] = accuracy_score(y_test, y_pred)  # Compute the accuracy

# Plot accuracy versus depth for different gamma values
for i in range(0, gammas.size):
    plt.plot(range(1, max_depth + 1), accuracies_RF[i], label=rf'$\gamma$={gammas[i]}')
plt.title(r'Accuracy vs Depth of RF for different $\gamma$ values')
plt.xlabel(r'$\gamma$')
plt.ylabel('Accuracy')
plt.legend()
plt.show()